In [3]:
pip install pandas rapidfuzz unidecode sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvid

In [21]:
import pandas as pd
import re
from unidecode import unidecode
from rapidfuzz import fuzz
from sentence_transformers import SentenceTransformer, util

# Load your Excel
df = pd.read_excel("/kaggle/input/dataset/Sample_100k.xlsx")

#  Normalize Arabic 
def normalize_arabic(text):
    text = re.sub(r'[ًٌٍَُِّْـ]', '', text)  # Remove diacritics
    text = re.sub(r'[إأآا]', 'ا', text)
    text = re.sub(r'ة', 'ه', text)
    text = re.sub(r'ى', 'ي', text)
    return text

# Universal Preprocessing 
def preprocess(name, lang):
    name = str(name).lower()
    name = re.sub(r'[^\w\s]', '', name)  # Remove punctuation
    name = re.sub(r'\s+', ' ', name).strip()
    
    if lang == "fr":
        name = unidecode(name)  # Remove accents (e.g. é → e)
    elif lang == "ar":
        name = normalize_arabic(name)
    
    words = name.split()
    return ' '.join(sorted(words))

# Preprocess names from both columns 
arabic_names = df["NOM_AR"].dropna().astype(str).tolist()
french_names = df["NOM_FR"].dropna().astype(str).tolist()

preprocessed_ar = [preprocess(n, "ar") for n in arabic_names]
preprocessed_fr = [preprocess(n, "fr") for n in french_names]

# Prepare embeddings (optional but powerful) 
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

ar_embeddings = model.encode(preprocessed_ar, convert_to_tensor=True)
fr_embeddings = model.encode(preprocessed_fr, convert_to_tensor=True)

#  Matching functions 





Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

100.0
 Too similar to an existing name (fuzzy match).


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 Semantically too close to an existing name.


In [29]:
def is_exact_duplicate(new_name, lang):
    clean = preprocess(new_name, lang)
    return clean in (preprocessed_ar if lang == "ar" else preprocessed_fr)

def is_fuzzy_duplicate(new_name, lang, threshold=85):
    clean = preprocess(new_name, lang)
    names = preprocessed_ar if lang == "ar" else preprocessed_fr
    for existing in names:
        if fuzz.token_set_ratio(clean, existing) >= threshold:
            print(existing)
            print(fuzz.token_set_ratio(clean, existing))
            return True
    return False

def is_semantically_similar(new_name, lang, threshold=0.85):
    clean = preprocess(new_name, lang)
    vec = model.encode(clean, convert_to_tensor=True)
    db = ar_embeddings if lang == "ar" else fr_embeddings
    return util.cos_sim(vec, db).max().item() > threshold

# Main Check Function 

def check_name_violation(name, lang):
    if is_exact_duplicate(name, lang):
        return " This name already exists (exact/reordered match)."
    if is_fuzzy_duplicate(name, lang):
        return " Too similar to an existing name (fuzzy match)."
    if is_semantically_similar(name, lang):
        return " Semantically too close to an existing name."
    return "✅ This name is acceptable."

In [30]:
print(check_name_violation("marketing center", "fr"))


center
100.0
 Too similar to an existing name (fuzzy match).


In [31]:
import pandas as pd
import re
from unidecode import unidecode
from rapidfuzz import fuzz
from sentence_transformers import SentenceTransformer, util

# Load your Excel
df = pd.read_excel("/kaggle/input/dataset/Sample_100k.xlsx")

#  Normalize Arabic 
def normalize_arabic(text):
    text = re.sub(r'[ًٌٍَُِّْـ]', '', text)  # Remove diacritics
    text = re.sub(r'[إأآا]', 'ا', text)
    text = re.sub(r'ة', 'ه', text)
    text = re.sub(r'ى', 'ي', text)
    return text

# Universal Preprocessing 
def preprocess(name, lang):
    name = str(name).lower()
    name = re.sub(r'[^\w\s]', '', name)  # Remove punctuation
    name = re.sub(r'\s+', ' ', name).strip()
    
    if lang == "fr":
        name = unidecode(name)  # Remove accents (e.g. é → e)
    elif lang == "ar":
        name = normalize_arabic(name)
    
    words = name.split()
    return ' '.join(sorted(words))

# Preprocess names from both columns 
arabic_names = df["NOM_AR"].dropna().astype(str).tolist()
french_names = df["NOM_FR"].dropna().astype(str).tolist()

preprocessed_ar = [preprocess(n, "ar") for n in arabic_names]
preprocessed_fr = [preprocess(n, "fr") for n in french_names]

# Prepare embeddings (optional but powerful) 
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

ar_embeddings = model.encode(preprocessed_ar, convert_to_tensor=True)
fr_embeddings = model.encode(preprocessed_fr, convert_to_tensor=True)

#  Matching functions 

def check_exact_match(name, lang):
    clean = preprocess(name, lang)
    db = preprocessed_ar if lang == "ar" else preprocessed_fr

    for existing in db:
        if clean == existing:
            return {
                "type": "exact",
                "match": existing,
                "score": 100,
                "details": "Same words (reordered or identical)"
            }
    return None

from rapidfuzz import fuzz, process

def check_fuzzy_match(name, lang, threshold=85):
    clean = preprocess(name, lang)
    db = preprocessed_ar if lang == "ar" else preprocessed_fr

    best_match, score, _ = process.extractOne(clean, db, scorer=fuzz.token_set_ratio)
    
    if score >= threshold:
        common_words = set(clean.split()).intersection(set(best_match.split()))
        return {
            "type": "fuzzy",
            "match": best_match,
            "score": score,
            "details": f"Shared words: {', '.join(common_words)}"
        }
    return None
def check_semantic_match(name, lang, threshold=0.85):
    clean = preprocess(name, lang)
    query_vec = model.encode(clean, convert_to_tensor=True)
    db_vecs = ar_embeddings if lang == "ar" else fr_embeddings
    db_names = preprocessed_ar if lang == "ar" else preprocessed_fr

    cos_scores = util.cos_sim(query_vec, db_vecs)[0]
    best_idx = cos_scores.argmax().item()
    best_score = cos_scores[best_idx].item()

    if best_score >= threshold:
        return {
            "type": "semantic",
            "match": db_names[best_idx],
            "score": round(best_score * 100, 2),
            "details": "High semantic similarity (meaning is close)"
        }
    return None

# Main Check Function 

def full_name_check(name, lang):
    checks = [
        check_exact_match(name, lang),
        check_fuzzy_match(name, lang),
        check_semantic_match(name, lang)
    ]
    
    for result in checks:
        if result:
            return result
    
    return {
        "type": "unique",
        "match": None,
        "score": 0,
        "details": "Name is sufficiently different from all existing ones"
    }
result = full_name_check("تكنو المستقبل", "ar")
print(result)

Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'type': 'fuzzy', 'match': 'المستقبل', 'score': 100.0, 'details': 'Shared words: المستقبل'}


In [32]:


def check_exact_match(name, lang):
    clean = preprocess(name, lang)
    db = preprocessed_ar if lang == "ar" else preprocessed_fr

    for existing in db:
        if clean == existing:
            return {
                "type": "exact",
                "match": existing,
                "score": 100,
                "details": "Same words (reordered or identical)"
            }
    return None

from rapidfuzz import fuzz, process

def check_fuzzy_match(name, lang, threshold=85):
    clean = preprocess(name, lang)
    db = preprocessed_ar if lang == "ar" else preprocessed_fr

    best_match, score, _ = process.extractOne(clean, db, scorer=fuzz.token_set_ratio)
    
    if score >= threshold:
        common_words = set(clean.split()).intersection(set(best_match.split()))
        return {
            "type": "fuzzy",
            "match": best_match,
            "score": score,
            "details": f"Shared words: {', '.join(common_words)}"
        }
    return "no matches found"
def check_semantic_match(name, lang, threshold=0.85):
    clean = preprocess(name, lang)
    query_vec = model.encode(clean, convert_to_tensor=True)
    db_vecs = ar_embeddings if lang == "ar" else fr_embeddings
    db_names = preprocessed_ar if lang == "ar" else preprocessed_fr

    cos_scores = util.cos_sim(query_vec, db_vecs)[0]
    best_idx = cos_scores.argmax().item()
    best_score = cos_scores[best_idx].item()

    if best_score >= threshold:
        return {
            "type": "semantic",
            "match": db_names[best_idx],
            "score": round(best_score * 100, 2),
            "details": "High semantic similarity (meaning is close)"
        }
    return None

# Main Check Function 

def full_name_check(name, lang):
    checks = [
        check_exact_match(name, lang),
        check_fuzzy_match(name, lang),
        #check_semantic_match(name, lang)
    ]
    
    for result in checks:
        if result:
            return result
    
    return {
        "type": "unique",
        "match": None,
        "score": 0,
        "details": "Name is sufficiently different from all existing ones"
    }
result = full_name_check("samar de   tourisme  ", "fr")
print(result)


{'type': 'fuzzy', 'match': 'de', 'score': 100.0, 'details': 'Shared words: de'}
